<a href="https://colab.research.google.com/github/ykitaguchi77/Colab_Scripts/blob/master/Convert%20Pytorch%20model%20to%20TFLite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Convert Pytorch models to TFLite**<br>
https://qiita.com/lain21/items/9f9f9707ebad4bbc627d
https://qiita.com/hirune924/items/06520f44927b0844a86c

In [2]:
#google driveをcolabolatoryにマウント
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Pytorch to ONNX

https://colab.research.google.com/gist/amahendrakar/6042511a73aa403109ff22452502ffc5/46285.ipynb#scrollTo=rDAI_Iqu7Ucx <br>
https://ichi.pro/pytorch-o-tensorflowlite-ni-henkansuru-watashi-no-tabi-14449401839341<br>
https://qiita.com/hirune924/items/06520f44927b0844a86c <br>

In [41]:
model_name = "resnet50"

In [42]:
import torch
import torchvision.models as models

dummy_input = torch.randn((1, 3, 224, 224))
model = models.resnet50()
torch.onnx.export(model, dummy_input, "converted.onnx", verbose=True, input_names=['input'], output_names=['output'])

graph(%input : Float(1, 3, 224, 224, strides=[150528, 50176, 224, 1], requires_grad=0, device=cpu),
      %fc.weight : Float(1000, 2048, strides=[2048, 1], requires_grad=1, device=cpu),
      %fc.bias : Float(1000, strides=[1], requires_grad=1, device=cpu),
      %497 : Float(64, 3, 7, 7, strides=[147, 49, 7, 1], requires_grad=0, device=cpu),
      %498 : Float(64, strides=[1], requires_grad=0, device=cpu),
      %500 : Float(64, 64, 1, 1, strides=[64, 1, 1, 1], requires_grad=0, device=cpu),
      %501 : Float(64, strides=[1], requires_grad=0, device=cpu),
      %503 : Float(64, 64, 3, 3, strides=[576, 9, 3, 1], requires_grad=0, device=cpu),
      %504 : Float(64, strides=[1], requires_grad=0, device=cpu),
      %506 : Float(256, 64, 1, 1, strides=[64, 1, 1, 1], requires_grad=0, device=cpu),
      %507 : Float(256, strides=[1], requires_grad=0, device=cpu),
      %509 : Float(256, 64, 1, 1, strides=[64, 1, 1, 1], requires_grad=0, device=cpu),
      %510 : Float(256, strides=[1], requir

#ONNX to Tensorflow

In [43]:
!pip install onnx_tf
from onnx_tf.backend import prepare
import onnx

TF_PATH = "./converted.pb" # where the representation of tensorflow model will be stored
ONNX_PATH = "./converted.onnx" # path to my existing ONNX model
onnx_model = onnx.load(ONNX_PATH)  # load onnx model

# prepare function converts an ONNX model to an internel representation
# of the computational graph called TensorflowRep and returns
# the converted representation.
tf_rep = prepare(onnx_model)  # creating TensorflowRep object

# export_graph function obtains the graph proto corresponding to the ONNX
# model associated with the backend representation and serializes
# to a protobuf file.
tf_rep.export_graph(TF_PATH)

INFO:tensorflow:Assets written to: ./converted.pb/assets


INFO:tensorflow:Assets written to: ./converted.pb/assets


#Tensorflow to TFLite

In [44]:
# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model(TF_PATH) # path to the SavedModel directory
converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
  tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]
tflite_model = converter.convert()

# Save the model.
with open(model_name +".tflite", "wb") as f:
  f.write(tflite_model)